# Trabajo Practico 1 - Fundamentos de los Helicópteros

## Introducción

El trabajo consiste en calcular las características de vuelo en punto fijo para un helicoptero convencional, asi como la distribución de velocidades y cargas a lo largo de las palas del rotor. El helicóptero en estudio es el Robison Beta R-22. Para determinar la potencia necesaria para mover el rotor y la tracción generada, para distintos ángulos de paso, empleamos la teoria combinada del elemento de pala y la cantidad de movimiento

### Datos de Entrada

In [ ]:
import sys
sys.path.append('C:\\Users\\enri\\PYTHON_PROJECTS\\Helicopteros')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import atmosfera_estandar
from scipy import interpolate
import helicopter as Helicopter
%matplotlib inline

In [ ]:
gravity = 9.81 #[m/s**2]
temp = 15 + 273 # Sea level temperature
height = [0, 1000.0, 2000.0, 3000.0, 4000.0]
density = [atmosfera_estandar.atmosfera_estandar('altura',h)[4] for h in height]
temperature = [atmosfera_estandar.atmosfera_estandar('altura',h)[3] for h in height]
vel_son = [atmosfera_estandar.atmosfera_estandar('altura',h)[6] for h in height]

In [ ]:
airfoil_name = 'NACA 0012'
a2d = 6 # [1/rad] airfoil Cl vs alpha slope
alpha_perdida = 9*np.pi/180.0
airfoil = Helicopter.Airfoil(name=airfoil_name, a2d=a2d, alpha_perdida=alpha_perdida)
blade_length = 7.7/2.0;
chord = 0.183 # [m]
twist = -7 * np.pi / 180.0 # [rad]
Fp = 0.97 # Blade's tip loss factor
Cd0 = 0.01035 # Mean Resistance coeficient
blade = Helicopter.Helicopter.Rotor.Blade(airfoil=airfoil, length=blade_length, chord=chord,
                               twist=twist, blade_tip_lost_factor=Fp, cd0=Cd0)
n_blades = 2
inertia_rotor = 36.4 #[kg m**2]
angular_velocity = 55.56 # [rad/s]
rotor = Helicopter.Helicopter.Rotor(blade=blade, number_of_blades=n_blades,
                         inertia=inertia_rotor, angular_velocity=angular_velocity)
weight = 590 # [kg]
helicopter = Helicopter.Helicopter(rotor=rotor, weight=weight)

pot_h_0 = 131 * 745.6998715822701 # [watt]
engine_manufacturer = Helicopter.EngineType.LYCOMING
helicopter.engine = Helicopter.Helicopter.Engine(engine_type=engine_manufacturer,
                                                potencia_maxima=pot_h_0)